# GAN - MNIST - 32×32 px
Generative Adversarial Network for generating images of handwritten digits - code for training GAN.

Developed by Daniel Konečný

## Initialize
Defines the basic libraries and initializes global variables needed in all codes. Connects the code to data source - Google Drive.

In [0]:
import numpy as np
import tensorflow as tf

dataset = 'mnist'
x_dimension = 32
y_dimension = 32
note = ''

project_name = f'{dataset}{x_dimension}x{y_dimension}{note}'
project_path = '/'
grid_path = f'{project_path}grids/'
model_path = f'{project_path}models/'

latent_dimension = 64
batch_size = 256

## Training functions
All functions necessary for training, initialize before.

In [0]:
import time
from tensorflow.keras import layers
from matplotlib import pyplot


def get_discriminator(image_shape=(x_dimension, y_dimension, 1)):
	discriminator = tf.keras.Sequential()
 
	discriminator.add(layers.Conv2D(32, (1, 1), padding='same', input_shape=image_shape))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	# None is the batch size.
	assert discriminator.output_shape == (None, x_dimension, y_dimension, 32)

	discriminator.add(layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//2, y_dimension//2, 32)
 
	discriminator.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//4, y_dimension//4, 64)
 
	discriminator.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
	discriminator.add(layers.LeakyReLU(alpha=0.2))
	discriminator.add(layers.Dropout(0.4))
	assert discriminator.output_shape == (None, x_dimension//8, y_dimension//8, 64)
 
	discriminator.add(layers.Flatten())
	discriminator.add(layers.Dense(1, activation='sigmoid'))
 
	discriminator_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	discriminator.compile(loss='binary_crossentropy',
	                      optimizer=discriminator_optimizer,
						  metrics=['accuracy'])
 
	return discriminator


def get_generator():
	generator = tf.keras.Sequential()
	
	generator.add(layers.Dense(x_dimension//8 * y_dimension//8 * 64, input_dim=latent_dimension))
	generator.add(layers.LeakyReLU(alpha=0.2))
	generator.add(layers.Reshape((x_dimension//8, y_dimension//8, 64)))
	assert generator.output_shape == (None, x_dimension//8, y_dimension//8, 64)

	generator.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//4, y_dimension//4, 64)

	generator.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension//2, y_dimension//2, 32)
 
	generator.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))
	generator.add(layers.LeakyReLU(alpha=0.2))
	assert generator.output_shape == (None, x_dimension, y_dimension, 32)
 
	generator.add(layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same'))
	assert generator.output_shape == (None, x_dimension, y_dimension, 1)

	return generator


def get_gan(generator, discriminator):
	gan = tf.keras.Sequential()
 
	gan.add(generator)
	discriminator.trainable = False
	gan.add(discriminator)

	gan_optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	gan.compile(loss='binary_crossentropy', optimizer=gan_optimizer)

	return gan


def get_dataset():
	(dataset_train, _), (dataset_test, _) = tf.keras.datasets.mnist.load_data()
	dataset_28 = np.concatenate((dataset_train, dataset_test))
	dataset_28 = dataset_28.astype('float32') / 255.0
	
	# Create numpy array of the new dimensions with zeros.
	dataset_32 = np.zeros((len(dataset_28), x_dimension, y_dimension))

	# Copy values.
	for image_index in range(len(dataset_28)):
		for row_index in range(len(dataset_28[0])):
			for column_index in range(len(dataset_28[0][0])):
				dataset_32[image_index][row_index+2][column_index+2] = \
				dataset_28[image_index][row_index][column_index]

	return np.expand_dims(dataset_32, axis=-1)


def get_latent_points(sample_count=1):
	latents = np.empty((sample_count, latent_dimension))

	for latents_index in range(sample_count):
		randoms = np.random.normal(0, 1, latent_dimension)
		normalizer = np.sum(randoms**2)**0.5
		latent = randoms/normalizer
		latents[latents_index] = latent
	
	return latents


def get_generated_images(generator, image_count):
	generated_latents = get_latent_points(image_count)
	generated_images = generator.predict(generated_latents)
	generated_labels = np.zeros((image_count, 1))
	return generated_images, generated_labels


def get_real_images(dataset, image_count):
	real_randoms = np.random.randint(0, dataset.shape[0], image_count)
	real_images = dataset[real_randoms]
	real_labels = np.ones((image_count, 1))
	return real_images, real_labels


def create_image_grid(examples, image_grid_size, epoch_index):
	for grid_index in range(image_grid_size * image_grid_size):
		pyplot.subplot(image_grid_size, image_grid_size, 1 + grid_index)
		pyplot.axis('off')
		pyplot.imshow(examples[grid_index, :, :, 0], cmap='gray_r')
	pyplot.savefig(f'{grid_path}{project_name}_grid{epoch_index+1:04d}.png')
	pyplot.show()


def evaluate_model(epoch_index, generator, discriminator, dataset,
                   sample_count=100, image_grid_size=5):
	real_images, real_labels = get_real_images(dataset, sample_count)
	_, real_accuracy = discriminator.evaluate(real_images, real_labels, verbose=0)
	
	generated_images, generated_labels = get_generated_images(generator, sample_count)
	_, generated_accuracy = discriminator.evaluate(generated_images, generated_labels, verbose=0)
	
	print(f'Accuracy real: {real_accuracy*100:.0f} %, generated: {generated_accuracy*100:.0f} %')
	generator.save(f'{model_path}{project_name}_generator{epoch_index+1:04d}.h5')
 
	if (epoch_index + 1) % 10 == 0:
		display_latent = get_latent_points(image_grid_size*image_grid_size)
		display_images = generator.predict(display_latent)
		create_image_grid(display_images, image_grid_size, epoch_index)


def train(generator, discriminator, gan, dataset, epoch_count):
	batch_count = dataset.shape[0] // batch_size

	for epoch_index in range(epoch_count):
		epoch_start_time = time.time()
  
		for batch_index in range(batch_count):
			# Discriminator training.
			real_images, real_labels = get_real_images(dataset, batch_size // 2)
			generated_images, generated_labels = get_generated_images(generator, batch_size // 2)
			images, labels = np.vstack((real_images, generated_images)), np.vstack((real_labels, generated_labels))
			discriminator_loss, _ = discriminator.train_on_batch(images, labels)
   
			# Generator training.
			generated_latent = get_latent_points(batch_size)
			generated_labels = np.ones((batch_size, 1))
			generator_loss = gan.train_on_batch(generated_latent, generated_labels)
   
			print(f'- {epoch_index+1}, {batch_index+1}/{batch_count}, d={discriminator_loss:.3f}, g={generator_loss:.3f}')
   
		evaluate_model(epoch_index, generator, discriminator, dataset)

		epoch_end_time = time.time()
		print(f'Epoch {epoch_index+1} finished in {epoch_end_time - epoch_start_time} s.')

## Training
Set epoch count and launch for training of the GAN.

In [0]:
epoch_count = 1000

print("Creating GAN...")
discriminator = get_discriminator()
generator = get_generator()
gan = get_gan(generator, discriminator)

print("Loading dataset...")
dataset = get_dataset()

print("Training...")
train(generator, discriminator, gan, dataset, epoch_count)

## Summary
Launch to display the summary of Discriminator and Generator models.

In [0]:
print('\x1b[0;32;40m' + 'DISCRIMINATOR' + '\x1b[0m')
discriminator = get_discriminator()
print(f'{discriminator.summary()}\n')

print('\x1b[0;32;40m' + 'GENERATOR' + '\x1b[0m')
generator = get_generator()
print(f'{generator.summary()}\n')